<a id='top_cell'></a>
# Power BI Report User Access Utilities

* [Find email for name(s)](#find_email)
* [View a person's report permissions](#get_artifiact_access)
* [Search for a report's workspace](#search_report_workspace)
* [Who has permissions to particular report](#get_report_access)

In [ ]:
# Import modules needed to make API calls
import requests
import json
import pandas as pd
import pyodbc

In [ ]:
connect_file = json.load(open('data_config_git.json'))
data_sources = connect_file['data_sources']['db_name']['connection_string']

In [ ]:
cnxn = pyodbc.connect(data_sources)
cursor = cnxn.cursor()

In [ ]:
# Graph & PBI API shared variables
access_token_url = 'https://login.microsoftonline.com/'
tenant_id = connect_file['data_sources']['common_api']['tenant_id']
full_token_url = access_token_url + tenant_id + '/oauth2/token'
headers_token = {'Content_Type': 'application/x-www-form-urlencoded'}
# Graph API variables
client_id_graph = connect_file['data_sources']['graph_api']['client_id_graph']
client_secret_graph = connect_file['data_sources']['graph_api']['client_secret_graph']
body_token_graph = {
    'grant_type': 'client_credentials',
    'client_id': client_id_graph,
    'client_secret': client_secret_graph,
    'resource': 'https://graph.microsoft.com'
    }    
headers_get_graph = ''
# PBI API variables
client_id_pbi = connect_file['data_sources']['pbi_api']['client_id_pbi']
client_secret_pbi = connect_file['data_sources']['pbi_api']['client_secret_pbi']
body_token_pbi = {
    'grant_type': 'client_credentials',
    'client_id': client_id_pbi,
    'client_secret': client_secret_pbi,
    'resource': 'https://analysis.windows.net/powerbi/api'
}
headers_get_pbi = ''

# Post for Graph API token
def post_for_bearer_api_graph():
    global headers_get_graph
    post_url = full_token_url
    response = requests.post(
        post_url, 
        headers = headers_token, 
        data = body_token_graph).json()
    access_token_graph = 'Bearer ' + response['access_token']
    headers_get_graph = {'Authorization': access_token_graph}
    headers_get_graph['ConsistencyLevel'] = 'eventual'
    return

# Get Graph API
def get_api_graph(base_url):
    if len(headers_get_graph) == 0: # authorization token is empty
        post_for_bearer_api_graph()
    response = requests.get(
        base_url, headers = headers_get_graph)
    return response.json()

# Post for Power BI API token
def post_for_bearer_api_pbi():
    global headers_get_pbi
    post_url = full_token_url
    response = requests.post(
        post_url, 
        headers = headers_token, 
        data = body_token_pbi).json()
    access_token_pbi = 'Bearer ' + response['access_token']
    headers_get_pbi = {'Authorization': access_token_pbi}
    return

# Get Power BI API
def get_api_pbi(base_url):
    if len(headers_get_pbi) == 0: # authorization token is empty
        post_for_bearer_api_pbi()
    response = requests.get(
        base_url, headers = headers_get_pbi)
    return response.json()

<a id='find_email'></a>
## For list of names, return email addresses
[Return to Top](#top_cell)

In [ ]:
# Provide string of names to look up, choose uniquest names
name_string = 'Alexander, McDonald, Lewis, Lester'
name_list = []
email_list = []
email_domain = '@domain.com'
name_list = name_string.split(',')
name_list = [s.strip() for s in name_list]

for n in name_list:
    base_url_names = 'https://graph.microsoft.com/v1.0/users?$search="displayName:' + n + '"'
    next_link = True
    call_number = 1
    while next_link == True:
        user_email = get_api_graph(base_url_names)
        try:
            base_url_names = user_email['@odata.nextLink']
            call_number += 1
        except:
            next_link = False
        for e in user_email['value']:
            if email_domain in e['mail'].lower():
                email_list.append(e['mail'])

email_list

In [ ]:
# Write list to string of email addresses to paste into Outlook
'; '.join(email_list)

<a id='get_artifiact_access'></a>
## Get User Artifact Access
[Return to Top](#top_cell)

In [ ]:
# String of email addresses to search for
email_string = 'jdoe@domain.com, clewis@domain.com'
email_list = []
email_list = email_string.split(',')
email_list = [s.strip() for s in email_list]
artifact_dict = {}

for e in email_list:
    base_url_artifact = 'https://api.powerbi.com/v1.0/myorg/admin/users/' \
        + e + '/artifactAccess'
    continuationUri = True
    call_number = 1
    artifact_dict[e] = {}
    while continuationUri == True:
        artifact_access = get_api_pbi(base_url_artifact)
        try:
            base_url_artifact = artifact_access['continuationUri']
            call_number += 1
        except:
            continuationUri = False
        for a in artifact_access['ArtifactAccessEntities']:
            artifact_dict[e].update({a['displayName']: a['artifactId']})

In [ ]:
artifact_dict

In [ ]:
# Create dataframe from artifact_dict
df = pd.DataFrame.from_dict(artifact_dict, orient = 'index')
df.index.name = 'email'
df.reset_index(inplace = True)
df1 = (df.set_index(["email"])
         .stack()
         .reset_index(name='artifactId')
         .rename(columns={'level_1':'report'}))
# df1

In [ ]:
# Search for specific report
report_name = 'Book Sales Report' # Report name to search for
report_found = df1.loc[df1['report'].str.contains(report_name, case=False)]
report_found

<a id='search_report_workspace'></a>
## Search for a Report's Workspace
Knowing that a user has access to <report name> is not enough given that some report names are repeated in multiple workspaces. The following cells will help you to discover which workspace holds the report that the user has access to.

Microsoft's API currently does not enable searching the tenant for an artifict id. These cells rely on a view built on a sql table logging the Power BI activity events. The PBI activity events logs activities such as report views and records the report and the containing workspace. The view groups these to get a list of all workspaces & their reports. Obviously this depends on the activity events having recorded views of for all reports.
    
https://learn.microsoft.com/en-us/rest/api/power-bi/admin/get-activity-events
    
Note that the artifactId might be the dataset OR report ID, thus the query searches both columns for the ID.

[Return to Top](#top_cell)

In [ ]:
# artifactId taken from previous cell
artifact_id = '<id>'
sql = """
    SELECT * 
    FROM dbo.v_PbiActivityEventLog
    WHERE ReportId = ?
      OR DatasetId = ?
    """
val = (artifact_id, artifact_id)
cursor.execute(sql, val)
rows = cursor.fetchall()
for row in rows:
    print(row.WorkSpaceName)    

<a id='get_report_access'></a>
## Who Has Report Access
The following blocks will get the users/groups for the app, and you will need to manually narrow down the user search to the appropriate group(s).

[Return to Top](#top_cell)

In [ ]:
# Get the list of Apps in the tenant
base_url_apps = 'https://api.powerbi.com/v1.0/myorg/admin/apps?$top=5000'
apps = get_api_pbi(base_url_apps)

In [ ]:
# Search for a particular App name to get ID by name
app_search_name = 'Book Sales Reporting' # Name of App in question
for i in apps['value']:
    if i['name'] == app_search_name:
        print('The id for App {name} is {id}'
              .format(name = i['name'], id = i['id']))
#         app_id = i['id']

In [ ]:
# Get users of this app ID
app_id = '<id>' # ID from previous cell
base_url_app_member = 'https://api.powerbi.com/v1.0/myorg/admin/apps/' \
    + app_id + '/users'
app_members = get_api_pbi(base_url_app_member)

In [ ]:
app_members

In [ ]:
# Write individual users' email to a list
member_email_list = []

for i in app_members['value']:
    if i['principalType'] != 'Group' and i['principalType'] != 'App':
        member_email_list.append(i['emailAddress'])
member_email_list

In [ ]:
# Return groups who have access to App
app_user_group_list = []
for i in app_members['value']:
    if i['principalType'] == 'Group':
        print(i['displayName'],i['graphId'])

In [ ]:
# Search groups for individual users & append to earlier list
group_id_list = [ # Target groups from previous cell
                '<id_1>',
                '<id_2>'
                ]

for g in group_id_list:
    base_url_groups = 'https://graph.microsoft.com/v1.0/groups/' \
        + g + '/members'
    next_link = True
    call_number = 1
    while next_link == True:
        group_user_email = get_api_graph(base_url_groups)
        try:
            base_url_groups = group_user_email['@odata.nextLink']
            call_number += 1
        except:
            next_link = False
        for e in group_user_email['value']:
            member_email_list.append(e['mail'])

member_email_list

In [ ]:
# Write list to string of email addresses for use in Outlook
'; '.join(member_email_list)